## Alchemifying stuff with `openmmtools`

I'm just doing this to query the alchemical system that the `AbsoluteAlchemicalFactory` makes and see what we have to modify in order to make it a valid sampler for perses...

In [133]:
from openmmtools import testsystems
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion

In [134]:
factory = AbsoluteAlchemicalFactory()
test = testsystems.AlanineDipeptideVacuum()
positions, topology, system = test.positions, test.topology, test.system

for example, i'll 'alchemify' the first 5 atoms in the vacuum alanine dipeptide system

In [135]:
alch_region = AlchemicalRegion(alchemical_atoms = range(6))

In [136]:
alch_sys = factory.create_alchemical_system(reference_system=system, alchemical_regions=alch_region)

In [137]:
system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7fc6516c0030> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7fc6516c0cf0> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7fc6516c0cc0> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7fc6516c0240> >,
 <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7fc6516c05a0> >]

Let's query the alchemical system to see what it looks like...

In [138]:
num_forces = alch_sys.getNumForces()
force_dict = {}
for idx, entry in enumerate(alch_sys.getForces()):
    force_dict[idx] = entry
for key, val in force_dict.items():
    print(key, val)
    

0 <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7fc6516c03f0> >
1 <simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7fc6516c0090> >
2 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7fc6516c0990> >
3 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7fc6516c0c30> >
4 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7fc6516c0ed0> >
5 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x7fc6516c0510> >
6 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x7fc6516c0330> >
7 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x7fc6516c0600> >
8 <simtk.o

we can remove force 0; let's take a look at the standard valence forces (1-3) to see if they include _all_ of the atom terms...

In [139]:
bond_force = force_dict[1]
for bond_idx in range(bond_force.getNumBonds()):
    print(bond_force.getBondParameters(bond_idx))

[4, 5, Quantity(value=0.12290000000000001, unit=nanometer), Quantity(value=476976.00000000006, unit=kilojoule/(nanometer**2*mole))]
[4, 6, Quantity(value=0.1335, unit=nanometer), Quantity(value=410032.00000000006, unit=kilojoule/(nanometer**2*mole))]
[1, 4, Quantity(value=0.1522, unit=nanometer), Quantity(value=265265.60000000003, unit=kilojoule/(nanometer**2*mole))]
[14, 15, Quantity(value=0.12290000000000001, unit=nanometer), Quantity(value=476976.00000000006, unit=kilojoule/(nanometer**2*mole))]
[14, 16, Quantity(value=0.1335, unit=nanometer), Quantity(value=410032.00000000006, unit=kilojoule/(nanometer**2*mole))]
[8, 10, Quantity(value=0.1526, unit=nanometer), Quantity(value=259408.00000000003, unit=kilojoule/(nanometer**2*mole))]
[8, 14, Quantity(value=0.1522, unit=nanometer), Quantity(value=265265.60000000003, unit=kilojoule/(nanometer**2*mole))]
[6, 8, Quantity(value=0.1449, unit=nanometer), Quantity(value=282001.60000000003, unit=kilojoule/(nanometer**2*mole))]
[16, 18, Quantit

and let's compare this to the original bond force...


In [140]:
og_bond_force= system.getForce(0)
for bond_idx in range(og_bond_force.getNumBonds()):
    print(og_bond_force.getBondParameters(bond_idx))

[4, 5, Quantity(value=0.12290000000000001, unit=nanometer), Quantity(value=476976.00000000006, unit=kilojoule/(nanometer**2*mole))]
[4, 6, Quantity(value=0.1335, unit=nanometer), Quantity(value=410032.00000000006, unit=kilojoule/(nanometer**2*mole))]
[1, 4, Quantity(value=0.1522, unit=nanometer), Quantity(value=265265.60000000003, unit=kilojoule/(nanometer**2*mole))]
[14, 15, Quantity(value=0.12290000000000001, unit=nanometer), Quantity(value=476976.00000000006, unit=kilojoule/(nanometer**2*mole))]
[14, 16, Quantity(value=0.1335, unit=nanometer), Quantity(value=410032.00000000006, unit=kilojoule/(nanometer**2*mole))]
[8, 10, Quantity(value=0.1526, unit=nanometer), Quantity(value=259408.00000000003, unit=kilojoule/(nanometer**2*mole))]
[8, 14, Quantity(value=0.1522, unit=nanometer), Quantity(value=265265.60000000003, unit=kilojoule/(nanometer**2*mole))]
[6, 8, Quantity(value=0.1449, unit=nanometer), Quantity(value=282001.60000000003, unit=kilojoule/(nanometer**2*mole))]
[16, 18, Quantit

so it appears that _all_ ofthe valence terms are copied over exactly. when we make the hybrid system (to allow for rest2), we will need to iterate through these and query the atom indices. 
1. if all atom indices are in alchemical region, then we need to place htem in a custombondforce that is scaled by the in_group lambda term. 
2. if all atom indices are outside of the alchemical region, then we need to leave the m in the standardbond force and not scale them
3. if the terms straddle the bonded region, we need to scale them with an inter_group lambda
such is the case for the angle force and the torsion force, too


now, let's take a look at the nonbonded force...

In [141]:
nb_force = force_dict[4]
for particle in range(nb_force.getNumParticles()):
    print(nb_force.getParticleParameters(particle))

[Quantity(value=0.0, unit=elementary charge), Quantity(value=0.26495327872602226, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[Quantity(value=0.0, unit=elementary charge), Quantity(value=0.33996695084507406, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[Quantity(value=0.0, unit=elementary charge), Quantity(value=0.26495327872602226, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[Quantity(value=0.0, unit=elementary charge), Quantity(value=0.26495327872602226, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[Quantity(value=0.0, unit=elementary charge), Quantity(value=0.33996695079448314, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[Quantity(value=0.0, unit=elementary charge), Quantity(value=0.2959921901644687, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[Quantity(value=-0.4157, unit=elementary charge), Quantity(value=0.3249998524031036, unit=nanometer), Quantity(value=0.7112799996555186, unit=kilojoule/mol

alright, so all of the particles in the alchemical region have zeroed parameters, so we should be able to leave this term as is; let's check the exceptions first

In [142]:
for exception in range(nb_force.getNumExceptions()):
    terms = nb_force.getExceptionParameters(exception)
    print(terms)

[5, 7, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=0.2014500181682605, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[4, 9, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=0.29355112752934986, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[3, 5, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=0.28047273444524545, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[3, 6, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=0.2949765655645629, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[2, 5, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=0.28047273444524545, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[2, 6, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=0.2949765655645629, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[1, 7, Quantity(value=0.0, unit=elementary charge**2), Quantity(value=0.22343739850856315, unit=nanomet

so it looks like the nbforce also zeroes the exceptions that live _inside_ the alchemical region _as well as_ the exceptions that straddle the alchemical region...again, this suggests we can just copy over the nonbonded force and leave it be...

In [143]:
nb_force.getNumExceptionParameterOffsets()

0

and just to be sure, there are no exception parameter offsets (good thing)

now, let's query the custom nonbonded forces (5-6) and see what those are about...

In [144]:
cnb_force = force_dict[5]
energy_expr= cnb_force.getEnergyFunction()
print(energy_expr)
num_params = cnb_force.getNumPerParticleParameters()
for idx in range(num_params):
    print(cnb_force.getPerParticleParameterName(idx))
for particle in range(cnb_force.getNumParticles()):
    terms = cnb_force.getParticleParameters(particle)
    print(terms)

U_electrostatics;U_electrostatics=((lambda_electrostatics)^softcore_d)*ONE_4PI_EPS0*chargeprod/reff_electrostatics;reff_electrostatics = sigma*((softcore_beta*(1.0-(lambda_electrostatics))^softcore_e + (r/sigma)^softcore_f))^(1/softcore_f);ONE_4PI_EPS0 = 138.935456;chargeprod = charge1*charge2;sigma = 0.5*(sigma1 + sigma2);
charge
sigma
(0.1123, 0.26495327872602226)
(-0.3662, 0.33996695084507406)
(0.1123, 0.26495327872602226)
(0.1123, 0.26495327872602226)
(0.5972000021951126, 0.33996695079448314)
(-0.5679000016463344, 0.2959921901644687)
(-0.4157, 0.3249998524031036)
(0.27190000000000003, 0.10690784617205229)
(0.033699999999999994, 0.33996695084507406)
(0.0823, 0.24713530426421654)
(-0.1825, 0.33996695084507406)
(0.06029999999999999, 0.26495327872602226)
(0.06029999999999999, 0.26495327872602226)
(0.06029999999999999, 0.26495327872602226)
(0.5973000005487781, 0.33996695079448314)
(-0.5679000016463344, 0.2959921901644687)
(-0.4157, 0.3249998524031036)
(0.27190000000000003, 0.10690784617

so it looks like these are electrostatics custom forces...we will have to loop through these, add another term called `is_alch` (whether is in alch region or not) and multiply the expression by a mixing term that scales the interaction appropriately if is in alchemical group or not...

In [145]:
excls = cnb_force.getNumExclusions()
for idx in range(excls):
    print(cnb_force.getExclusionParticles(idx))

[5, 7]
[4, 9]
[3, 5]
[3, 6]
[2, 5]
[2, 6]
[1, 7]
[0, 5]
[0, 6]
[15, 17]
[14, 19]
[14, 20]
[14, 21]
[13, 14]
[12, 14]
[11, 14]
[9, 11]
[9, 12]
[9, 13]
[9, 15]
[9, 16]
[8, 17]
[7, 9]
[7, 10]
[7, 14]
[6, 11]
[6, 12]
[6, 13]
[17, 19]
[17, 20]
[17, 21]
[5, 8]
[4, 10]
[4, 14]
[1, 8]
[15, 18]
[10, 15]
[10, 16]
[8, 18]
[6, 15]
[6, 16]
[0, 1]
[0, 2]
[0, 3]
[0, 4]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[1, 6]
[2, 3]
[2, 4]
[3, 4]
[4, 5]
[4, 6]
[4, 7]
[4, 8]
[5, 6]
[6, 7]
[6, 8]
[6, 9]
[6, 10]
[6, 14]
[7, 8]
[8, 9]
[8, 10]
[8, 11]
[8, 12]
[8, 13]
[8, 14]
[8, 15]
[8, 16]
[9, 10]
[9, 14]
[10, 11]
[10, 12]
[10, 13]
[10, 14]
[11, 12]
[11, 13]
[12, 13]
[14, 15]
[14, 16]
[14, 17]
[14, 18]
[15, 16]
[16, 17]
[16, 18]
[16, 19]
[16, 20]
[16, 21]
[17, 18]
[18, 19]
[18, 20]
[18, 21]
[19, 20]
[19, 21]
[20, 21]


the force also seems to be removing all in and outgroup normal exceptions, as well as the intergroup exceptions. it also adds exceptions between all intra group atoms. i.e. only intergroup interactions are working here... 

i am going to go out on a whim and say that force 6 is the _exact_ treatment as force 5 except it is for sterics forces...

now let's take a look at forces 7 and 8, which are custombond forces

In [146]:
cbf = force_dict[7]
print(cbf.getEnergyFunction())
for bond in range(cbf.getNumBonds()):
    print(cbf.getBondParameters(bond))

U_electrostatics;U_electrostatics=((lambda_electrostatics)^softcore_d)*ONE_4PI_EPS0*chargeprod/reff_electrostatics;reff_electrostatics = sigma*((softcore_beta*(1.0-(lambda_electrostatics))^softcore_e + (r/sigma)^softcore_f))^(1/softcore_f);ONE_4PI_EPS0 = 138.935456;
[5, 7, (-0.12867667537303196, 0.2014500181682605)]
[4, 9, (0.04095796681721481, 0.29355112752934986)]
[3, 6, (-0.03890259166666667, 0.2949765655645629)]
[2, 6, (-0.03890259166666667, 0.2949765655645629)]
[1, 7, (-0.08297481666666669, 0.22343739850856315)]
[0, 6, (-0.03890259166666667, 0.2949765655645629)]
[5, 8, (-0.015948525046234556, 0.3179795705047714)]
[4, 10, (-0.09082416700050672, 0.33996695081977857)]
[4, 14, (0.29725630136572584, 0.33996695079448314)]
[1, 8, (-0.010284116666666666, 0.33996695084507406)]


In [147]:
cbf = force_dict[8]
print(cbf.getEnergyFunction())
for bond in range(cbf.getNumBonds()):
    print(cbf.getBondParameters(bond))

U_electrostatics;U_electrostatics=((lambda_electrostatics)^softcore_d)*ONE_4PI_EPS0*chargeprod/reff_electrostatics;reff_electrostatics = sigma*((softcore_beta*(1.0-(lambda_electrostatics))^softcore_e + (r/sigma)^softcore_f))^(1/softcore_f);ONE_4PI_EPS0 = 138.935456;
[3, 5, (-0.053145975154069464, 0.28047273444524545)]
[2, 5, (-0.053145975154069464, 0.28047273444524545)]
[0, 5, (-0.053145975154069464, 0.28047273444524545)]


aha, so this is the _inter_ group electrostatics exception force, and the _intra_ group electrostatics exception force...

now, let's look at the customnonbonded forces 9 and 10 (probably electrostatic and steric, respectively)


In [148]:
cnb_force = force_dict[9]
energy_expr= cnb_force.getEnergyFunction()
print(energy_expr)
num_params = cnb_force.getNumPerParticleParameters()
for idx in range(num_params):
    print(cnb_force.getPerParticleParameterName(idx))
for particle in range(cnb_force.getNumParticles()):
    terms = cnb_force.getParticleParameters(particle)
    print(terms)

U_sterics;U_sterics = ((lambda_sterics)^softcore_a)*4*epsilon*x*(x-1.0);x = (sigma/reff_sterics)^6;reff_sterics = sigma*((softcore_alpha*(1.0-(lambda_sterics))^softcore_b + (r/sigma)^softcore_c))^(1/softcore_c);epsilon = sqrt(epsilon1*epsilon2);sigma = 0.5*(sigma1 + sigma2);
sigma
epsilon
(0.26495327872602226, 0.06568880010977264)
(0.33996695084507406, 0.45772959964740484)
(0.26495327872602226, 0.06568880010977264)
(0.26495327872602226, 0.06568880010977264)
(0.33996695079448314, 0.35982400053705343)
(0.2959921901644687, 0.8786399993381852)
(0.3249998524031036, 0.7112799996555186)
(0.10690784617205229, 0.06568880001765333)
(0.33996695084507406, 0.45772959964740484)
(0.24713530426421654, 0.0656888004119626)
(0.33996695084507406, 0.45772959964740484)
(0.26495327872602226, 0.06568880010977264)
(0.26495327872602226, 0.06568880010977264)
(0.26495327872602226, 0.06568880010977264)
(0.33996695079448314, 0.35982400053705343)
(0.2959921901644687, 0.8786399993381852)
(0.3249998524031036, 0.711279

In [149]:
excls = cnb_force.getNumExclusions()
for idx in range(excls):
    print(cnb_force.getExclusionParticles(idx))

[5, 7]
[4, 9]
[3, 5]
[3, 6]
[2, 5]
[2, 6]
[1, 7]
[0, 5]
[0, 6]
[15, 17]
[14, 19]
[14, 20]
[14, 21]
[13, 14]
[12, 14]
[11, 14]
[9, 11]
[9, 12]
[9, 13]
[9, 15]
[9, 16]
[8, 17]
[7, 9]
[7, 10]
[7, 14]
[6, 11]
[6, 12]
[6, 13]
[17, 19]
[17, 20]
[17, 21]
[5, 8]
[4, 10]
[4, 14]
[1, 8]
[15, 18]
[10, 15]
[10, 16]
[8, 18]
[6, 15]
[6, 16]
[0, 1]
[0, 2]
[0, 3]
[0, 4]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[1, 6]
[2, 3]
[2, 4]
[3, 4]
[4, 5]
[4, 6]
[4, 7]
[4, 8]
[5, 6]
[6, 7]
[6, 8]
[6, 9]
[6, 10]
[6, 14]
[7, 8]
[8, 9]
[8, 10]
[8, 11]
[8, 12]
[8, 13]
[8, 14]
[8, 15]
[8, 16]
[9, 10]
[9, 14]
[10, 11]
[10, 12]
[10, 13]
[10, 14]
[11, 12]
[11, 13]
[12, 13]
[14, 15]
[14, 16]
[14, 17]
[14, 18]
[15, 16]
[16, 17]
[16, 18]
[16, 19]
[16, 20]
[16, 21]
[17, 18]
[18, 19]
[18, 20]
[18, 21]
[19, 20]
[19, 21]
[20, 21]


ok so i was wrong...these are sterics and electrostatics, respectively. it looks like (based on the exceptions) these terms add the interactions between the ingroup and outgroup atoms...let's see if there are interaction groups

In [150]:
for i in range(cnb_force.getNumInteractionGroups()):
    print(cnb_force.getInteractionGroupParameters(i))

[(6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21), (0, 1, 2, 3, 4, 5)]


yeps...that's right

now what are forces 11 and 12?

In [151]:
cbf = force_dict[11]
print(cbf.getEnergyFunction())
for i in range(cbf.getNumBonds()):
    print(cbf.getBondParameters(i))

U_sterics;U_sterics = ((lambda_sterics)^softcore_a)*4*epsilon*x*(x-1.0);x = (sigma/reff_sterics)^6;reff_sterics = sigma*((softcore_alpha*(1.0-(lambda_sterics))^softcore_b + (r/sigma)^softcore_c))^(1/softcore_c);
[5, 7, (0.2014500181682605, 0.12012161254748986)]
[4, 9, (0.29355112752934986, 0.07687068191890915)]
[3, 6, (0.2949765655645629, 0.10807766850678557)]
[2, 6, (0.2949765655645629, 0.10807766850678557)]
[1, 7, (0.22343739850856315, 0.08670021357153467)]
[0, 6, (0.2949765655645629, 0.10807766850678557)]
[5, 8, (0.3179795705047714, 0.31708813251652274)]
[4, 10, (0.33996695081977857, 0.20291752986703307)]
[4, 14, (0.33996695079448314, 0.17991200026852672)]
[1, 8, (0.33996695084507406, 0.22886479982370242)]


In [152]:
cbf = force_dict[12]
print(cbf.getEnergyFunction())
for i in range(cbf.getNumBonds()):
    print(cbf.getBondParameters(i))

U_sterics;U_sterics = ((lambda_sterics)^softcore_a)*4*epsilon*x*(x-1.0);x = (sigma/reff_sterics)^6;reff_sterics = sigma*((softcore_alpha*(1.0-(lambda_sterics))^softcore_b + (r/sigma)^softcore_c))^(1/softcore_c);lambda_sterics=1.0;
[3, 5, (0.28047273444524545, 0.12012161263171671)]
[2, 5, (0.28047273444524545, 0.12012161263171671)]
[0, 5, (0.28047273444524545, 0.12012161263171671)]


aha, so this is the _inter_ group sterics exception force, and the _intra_ group sterics exception force...

## Conclusion
so now that we know what forces govern what interactions, it can be easily said that if we can create a hybrid force (for each endstate) that has all of the valence terms on and the appropriate nonbonded term turned on, then we can throw it into an `AbsoluteAlchemicalFactory` so that we may perform blues.<br>
Afterwards, if we want to do blest, then all we have to do is iterate through all ofthe custom forces and add a modifying expression that scales inter, and intra alchemical region atoms accordingly (along with an int into each per particle or per bond parameter). <br>
This seems _very_ doable. Furthermore, i am pretty convinced that (in the future) we may be able to rewrite the modified Absolute factory in such a way that it allows for the _full_ relative partition and alchemical interpolation.